In [ ]:
import json
import pandas as pd
import pickle

In [ ]:
with open('proj5_params.json', 'r') as filename:
    parameters = json.load(filename)

## Exercise 1

In [4]:
ex01_df = pd.read_csv('proj5_timeseries.csv')
ex01_df.columns = ex01_df.columns.str.lower()
ex01_df.columns = ex01_df.columns.str.replace('[^a-zA-Z]', '_', regex=True)
ex01_df.head()

,date,consumption,wind,solar,wind_solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [5]:
first_column = ex01_df.columns[0]
ex01_df[first_column] = pd.to_datetime(ex01_df[first_column])
ex01_df = ex01_df.set_index(first_column)
original_frequency = parameters['original_frequency']
ex01_df.head()

,consumption,wind,solar,wind_solar
date,,,,
2006-01-01,1069.184,NaN,NaN,NaN
2006-01-02,1380.521,NaN,NaN,NaN
2006-01-03,1442.533,NaN,NaN,NaN
2006-01-04,1457.217,NaN,NaN,NaN
2006-01-05,1477.131,NaN,NaN,NaN


In [6]:
ex01_df.index

DatetimeIndex(['2006-01-01', '2006-01-02', '2006-01-03', '2006-01-04',
               '2006-01-05', '2006-01-06', '2006-01-07', '2006-01-08',
               '2006-01-09', '2006-01-10',
               ...
               '2017-12-22', '2017-12-23', '2017-12-24', '2017-12-25',
               '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29',
               '2017-12-30', '2017-12-31'],
              dtype='datetime64[ns]', name='date', length=4383, freq=None)

In [7]:
ex01_df = ex01_df.asfreq(original_frequency)

In [8]:
ex01_df.index

DatetimeIndex(['2006-01-01', '2006-01-02', '2006-01-03', '2006-01-04',
               '2006-01-05', '2006-01-06', '2006-01-07', '2006-01-08',
               '2006-01-09', '2006-01-10',
               ...
               '2017-12-22', '2017-12-23', '2017-12-24', '2017-12-25',
               '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29',
               '2017-12-30', '2017-12-31'],
              dtype='datetime64[ns]', name='date', length=4383, freq='D')

In [9]:
ex01_df.to_pickle('proj5_ex01.pkl')

# Exercise 2

In [10]:
ex02_df = ex01_df.copy()
print(ex02_df.head(5))
target_frequency = parameters['target_frequency']
print(target_frequency)
ex02_df = ex02_df.asfreq(target_frequency)
print(ex02_df.index)
ex02_df.to_pickle('proj5_ex02.pkl')

            consumption  wind  solar  wind_solar
date                                            
2006-01-01     1069.184   NaN    NaN         NaN
2006-01-02     1380.521   NaN    NaN         NaN
2006-01-03     1442.533   NaN    NaN         NaN
2006-01-04     1457.217   NaN    NaN         NaN
2006-01-05     1477.131   NaN    NaN         NaN
w
DatetimeIndex(['2006-01-01', '2006-01-08', '2006-01-15', '2006-01-22',
               '2006-01-29', '2006-02-05', '2006-02-12', '2006-02-19',
               '2006-02-26', '2006-03-05',
               ...
               '2017-10-29', '2017-11-05', '2017-11-12', '2017-11-19',
               '2017-11-26', '2017-12-03', '2017-12-10', '2017-12-17',
               '2017-12-24', '2017-12-31'],
              dtype='datetime64[ns]', name='date', length=627, freq='W-SUN')


## Exercise 3

In [11]:
print(parameters)

{'original_frequency': 'd', 'target_frequency': 'w', 'downsample_periods': 3, 'downsample_units': 'd', 'upsample_periods': 2, 'upsample_units': 'h', 'interpolation': 'polynomial', 'interpolation_order': 3, 'sensors_periods': 10, 'sensors_units': 's'}


In [12]:
downsample_periods = parameters['downsample_periods']
downsample_units = parameters['downsample_units']
print(downsample_periods, downsample_units)

3 d


In [13]:
ex03_df = ex01_df.resample(f'{downsample_periods}{downsample_units}').sum(min_count=downsample_periods)
ex03_df

,consumption,wind,solar,wind_solar
date,,,,
2006-01-01,3892.23800,NaN,NaN,NaN
2006-01-04,4337.77500,NaN,NaN,NaN
2006-01-07,4037.59500,NaN,NaN,NaN
2006-01-10,4707.36700,NaN,NaN,NaN
2006-01-13,4170.42200,NaN,NaN,NaN
...,...,...,...,...
2017-12-17,4462.39225,484.667,37.310,521.977
2017-12-20,4503.29557,578.692,26.177,604.869
2017-12-23,3525.21153,2148.306,34.164,2182.470


In [14]:
ex03_df.to_pickle('proj5_ex03.pkl')

## Exercise 4

In [15]:
upsample_periods = parameters['upsample_periods']
upsample_units = parameters['upsample_units']
interpolation = parameters['interpolation']
interpolation_order = parameters['interpolation_order']
print(upsample_periods, upsample_units, interpolation, interpolation_order)

2 h polynomial 3


In [16]:
ex04_df = ex01_df.resample(f'{upsample_periods}{upsample_units}').interpolate(interpolation, order=interpolation_order)
print(ex04_df.index.freq / ex01_df.index.freq)
ex04_df = ex04_df * (ex04_df.index.freq / ex01_df.index.freq)

0.08333333333333333


In [17]:
ex04_df

,consumption,wind,solar,wind_solar
date,,,,
2006-01-01 00:00:00,89.098667,NaN,NaN,NaN
2006-01-01 02:00:00,92.515716,NaN,NaN,NaN
2006-01-01 04:00:00,95.668110,NaN,NaN,NaN
2006-01-01 06:00:00,98.566792,NaN,NaN,NaN
2006-01-01 08:00:00,101.222704,NaN,NaN,NaN
...,...,...,...,...
2017-12-30 16:00:00,95.117424,63.173803,0.423908,63.597711
2017-12-30 18:00:00,94.371167,62.769170,0.626352,63.395522
2017-12-30 20:00:00,93.643499,62.132939,0.897118,63.030057


In [18]:
ex04_df.to_pickle('proj5_ex04.pkl')

## Exercise 5

In [21]:
sensors_periods = parameters['sensors_periods']
sensors_units = parameters['sensors_units']
print(sensors_periods, sensors_units)

10 s


In [22]:
ex05_df = pd.read_pickle('proj5_sensors.pkl')
ex05_df

,device_id,value
timestamp,,
2022-11-25 00:00:25,25,6.693750
2022-11-25 00:01:25,25,6.725000
2022-11-25 00:02:25,25,6.756250
2022-11-25 00:03:25,25,6.787500
2022-11-25 00:04:25,25,6.818750
...,...,...
2022-11-27 23:55:29,47,6.022222
2022-11-27 23:56:29,47,6.016667
2022-11-27 23:57:29,47,6.011111


In [23]:
ex05_df = ex05_df.pivot(columns='device_id', values='value')
ex05_df

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-11-25 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:04,NaN,NaN,NaN,5.6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.133333,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-27 23:59:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.584211,NaN,NaN,NaN,NaN
2022-11-27 23:59:51,NaN,NaN,NaN,NaN,NaN,6.094737,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
new_index = pd.date_range(ex05_df.index.round(f'{sensors_periods}{sensors_units}').min(), ex05_df.index.round(f'{sensors_periods}{sensors_units}').max(), freq=f'{sensors_periods}{sensors_units}')
ex05_df = ex05_df.reindex(new_index.union(ex05_df.index)).interpolate()
ex05_df

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:04,NaN,NaN,NaN,5.600000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:08,NaN,NaN,NaN,5.599533,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25 00:00:09,NaN,NaN,NaN,5.599066,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:50,4.305263,4.482353,5.296078,4.721053,6.380952,6.094932,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0
2022-11-27 23:59:51,4.305263,4.482353,5.298039,4.721053,6.380952,6.094737,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0
2022-11-27 23:59:55,4.305263,4.482353,5.300000,4.721053,6.380952,6.094737,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0
2022-11-27 23:59:59,4.305263,4.482353,5.300000,4.721053,6.380952,6.094737,5.155455,5.190476,4.24,5.888235,...,5.6,4.515789,2.3,5.133333,5.3,5.584211,4.7,6.2,4.005882,6.0


In [25]:
ex05_df = ex05_df.reindex(new_index)
ex05_df = ex05_df.dropna()
ex05_df

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:01:00,6.711111,5.109630,5.192130,5.588325,6.758754,5.562434,5.437475,7.551852,6.749673,5.333951,...,7.077160,6.500000,3.245833,7.106173,7.718301,5.215595,7.022896,7.110136,6.780507,7.733333
2022-11-25 00:01:10,6.716898,5.124444,5.203704,5.585990,6.764646,5.571252,5.437222,7.555967,6.751852,5.318519,...,7.080247,6.500000,3.248148,7.112346,7.722658,5.235088,7.026263,7.114035,6.775634,7.735859
2022-11-25 00:01:20,6.721528,5.136296,5.212963,5.584122,6.769360,5.578307,5.437020,7.559259,6.753595,5.306173,...,7.082716,6.500000,3.250000,7.117284,7.726144,5.250682,7.028956,7.117154,6.771735,7.737879
2022-11-25 00:01:30,6.729630,5.157037,5.229167,5.580853,6.777609,5.590653,5.436667,7.565021,6.756645,5.290741,...,7.087037,6.500000,3.253241,7.125926,7.732244,5.277973,7.033670,7.122612,6.764912,7.741414
2022-11-25 00:01:40,6.733102,5.165926,5.236111,5.579452,6.781145,5.595944,5.436515,7.567490,6.757952,5.285185,...,7.088889,6.500000,3.254630,7.129630,7.734858,5.289669,7.035690,7.124951,6.761988,7.742929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:20,4.304678,4.484096,5.268627,4.721053,6.380952,6.097661,5.159226,5.188889,4.248889,5.887800,...,5.600000,4.515789,2.300000,5.149383,5.300000,5.576023,4.697856,6.203472,4.005882,6.001235
2022-11-27 23:59:30,4.305263,4.482571,5.282353,4.721053,6.380952,6.096296,5.155455,5.190123,4.243704,5.888235,...,5.600000,4.515789,2.300000,5.140741,5.300000,5.580117,4.699220,6.200000,4.005882,6.000000
2022-11-27 23:59:40,4.305263,4.482353,5.288235,4.721053,6.380952,6.095712,5.155455,5.190476,4.241481,5.888235,...,5.600000,4.515789,2.300000,5.137037,5.300000,5.581871,4.699805,6.200000,4.005882,6.000000
2022-11-27 23:59:50,4.305263,4.482353,5.296078,4.721053,6.380952,6.094932,5.155455,5.190476,4.240000,5.888235,...,5.600000,4.515789,2.300000,5.133333,5.300000,5.584211,4.700000,6.200000,4.005882,6.000000


In [50]:
ex05_df.to_pickle('proj5_ex05.pkl')